In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

In [46]:
###Question 1: How do you handle duplicate values in a dataset in Python?
test_data = {
        'first_name': ['Jason', 'Jason', 'Jason','Tina', 'Jake', 'Amy'], 
        'last_name': ['Miller', 'Miller', 'Miller','Ali', 'Milner', 'Cooze'], 
        'age': [42, 42, 1111111, 36, 24, 73], 
        'preTestScore': [4, 4, 4, 31, 2, 3],
        'postTestScore': [25, 25, 25, 57, 62, 70]
}
df = pd.DataFrame(test_data, columns = ['first_name', 'last_name', 'age', 'preTestScore', 'postTestScore'])
df

,first_name,last_name,age,preTestScore,postTestScore
0,Jason,Miller,42,4,25
1,Jason,Miller,42,4,25
2,Jason,Miller,1111111,4,25
3,Tina,Ali,36,31,57
4,Jake,Milner,24,2,62
5,Amy,Cooze,73,3,70


In [48]:
#Identify which observations are duplicates
df.duplicated()

0    False
1     True
2    False
3    False
4    False
5    False
dtype: bool

In [59]:
#Drop duplicates in the first name column, and do not keep anything related to duplicates
df.drop_duplicates(['first_name'], keep=False)

,first_name,last_name,age,preTestScore,postTestScore
3,Tina,Ali,36,31,57
4,Jake,Milner,24,2,62
5,Amy,Cooze,73,3,70


In [ ]:
### Question 2: You are given two tables: friend_request and request_accepted.
### 1) friend_request contains requester_id, time and sent_to_id
### 2) request_accepted table contains time, acceptor_id and requestor_id. 
### How will you determine the overall acceptance rate of requests? 
##Define:overall acceptance rate=(# of unique acceptor_id in request_accepted table)/(# of unique sent_to_id in friend_request table)
#Python
friend_request_df = pd.DataFrame(friend_request)[subset=['requester_id', 'sent_to_id']].drop_duplicates()
request_accepted_df = pd.DataFrame(request_accepted)[subset=['acceptor_id', 'requester_id']].drop_duplicates()
acceptance_records_df=pd.merge(friend_request_df,request_accepted_df,how='left',on='requester_id')
acceptance_rate=acceptance_records_df.count()['acceptor_id']/ acceptance_records_df.count()['sent_to_id'] 
print("Acceptance Rate *100 = " + "%")
#SQL
#'SELECT COUNT(acceptor_id)  / COUNT(sent_to_id) AS acceptance_rate '
#            FROM friend_request_table fr 
#            LEFT JOIN request_accepted_table ra
#            ON fr.requester_id = ra.requester_id)

In [61]:
#Similarly to PrintDailyTimeSeries() in the sample code, write a Python function called PrintCumulativeTimeSeries(). Specifically:
#The function takes three inputs:
#  experiment_id
#  country
#  vertical
#The function produces two outputs:
#A time series chart showing the cumulative uplift (hint: search for cumsum)
#Print out the cumulative % uplift on revenue, experiment vs. control. Hint: (red - blue) / blue %

file_location = "/Users/Jeff/Documents/BittigerBA501ipython/"
file_name = "BA501_python_data.csv"
my_data = pd.read_csv(file_location + file_name)
my_data.head()

,country,date,experiment_id,order_id,spend_usd,user_id,vertical
0,JP,20170402,12624548,5813140433216629693,16.138855,5006730028073359543,ANDROID_APPS
1,US,20170403,12624549,16333056276711034931,9.990000,5938685669571545259,ANDROID_APPS
2,JP,20170404,12624548,11227119161741298366,19.887087,17623708458399141713,ANDROID_APPS
3,US,20170403,12624548,11927299833124210932,19.990000,18042812344456618148,ANDROID_APPS
4,JP,20170403,12624549,11644727017708872749,88.220093,4325104700699531031,ANDROID_APPS


In [100]:
def GetSeries(my_data,my_id,my_country,my_vertical):
    table = my_data[["date", "spend_usd"]][(my_data.experiment_id == my_id) & (my_data.country == my_country) & (my_data.vertical == my_vertical)]
    series = my_data.groupby('date')['spend_usd'].sum().cumsum()
    return(series)

In [101]:
def PlotSeries(index_series, experiment_series, control_series):
    df = pd.DataFrame(
        {
            "experiment": experiment_series.tolist(),
            "control": control_series.tolist()
        },
        index = index_series.tolist(), 
        columns = ["experiment", "control"]
    )
    df.plot()

In [102]:
def PrintCumulativeTimeSeries(my_data,experiment_ids, country, vertical):
    date_series = my_data.date
    index_series = np.unique(date_series) 
    index_series = np.sort(index_series)
    
    my_data = my_data.sort_values("date", ascending=True, inplace=False)
    experiment_series = GetSeries(my_data, experiment_ids[0], country, vertical)
    control_series = GetSeries(my_data, experiment_ids[1], country, vertical)
    
    PlotSeries(index_series, experiment_series, control_series)

In [105]:
PrintCumulativeTimeSeries(my_data,[12624549,12624548], "AR", "ANDROID_APPS")